# Подготовка данных
Будем использовать скрипт `prepare_messages.py`, предоставленный в задании.

In [1]:
!git clone https://github.com/solemn-leader/sirius-test

Cloning into 'sirius-test'...

remote: Enumerating objects: 27, done.

remote: Counting objects: 100% (27/27), done.

remote: Compressing objects: 100% (25/25), done.

remote: Total 27 (delta 6), reused 1 (delta 0), pack-reused 0

Receiving objects: 100% (27/27), 793.30 KiB | 6.10 MiB/s, done.

Resolving deltas: 100% (6/6), done.


In [2]:
!cd chat_export && unzip result.zip

Archive:  result.zip

  inflating: result.json             


In [3]:
!python sirius-test/prepare_messages.py --tg-history-path 'chat_export/result.json' --output-path 'data.csv'

Проверим, что данные преобразованы корректно:

In [1]:
# Для запуска на Kaggle
!cp /kaggle/input/scala-chat/data.csv data.csv

In [2]:
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,context_3,context_2,context_1,response
0,NaN,NaN,NaN,Не смог разгадать ребус
1,NaN,NaN,Не смог разгадать ребус,Намек на то что мы не такие но вроде никто бун...
2,NaN,NaN,NaN,в группе все андрофонеры и ты позвал джей ли\n...
3,NaN,NaN,в группе все андрофонеры и ты позвал джей ли\n...,еще чат офтопа\nтут с телеками за 300?
4,NaN,NaN,NaN,"все скатится в говно, если сюда зайдут решетка..."


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 25811 entries, 0 to 25810

Data columns (total 4 columns):

 #   Column     Non-Null Count  Dtype 

---  ------     --------------  ----- 

 0   context_3  4613 non-null   object

 1   context_2  8219 non-null   object

 2   context_1  16273 non-null  object

 3   response   25740 non-null  object

dtypes: object(4)

memory usage: 806.7+ KB


# Fine-tuning
## Загрузка модели

In [3]:
!pip install -q 'transformers[torch]'

In [6]:
from transformers import (AutoTokenizer, AutoModelWithLMHead,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
# Для запуска на Kaggle
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


## Проверка работы модели

In [9]:
import re

def extract_response(model_output: str) -> str:
    return re.sub(r'(@@[А-Я]+@@)+', '\n>>> ', model_output) + '\n'

In [14]:
def get_answers(question, model):
    print('Question:', question)
    prompt = f'@@ПЕРВЫЙ@@ {question} @@ВТОРОЙ@@'

    inputs = tokenizer(prompt, return_tensors='pt')
    generated_token_ids = model.generate(
        **inputs,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=3,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.2,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50261,
        max_new_tokens=100
    )
    context_with_response = [extract_response(tokenizer.decode(sample_token_ids)) for sample_token_ids in generated_token_ids]
    print('Responses:', *context_with_response, sep='\n')

# Проверим на вопросе по будущей тематике бота
get_answers('что такое scala?', model)

Setting `pad_token_id` to `eos_token_id`:50261 for open-end generation.


Question: что такое scala?
Responses:
что такое scala?
>>> Scala - язык программирования. Scala это язык разметки, который используется в веб-разработке.
>>> Спасибо большое!
>>> Пожалуйста. Обращайся, если что. :) P.S. А что за сайт-то? :) И почему ты решил, что это программирование? :D Или это не так работает? О_о
>>> Я не знаю, как это работает, я просто предположил. Ну и, по-моему, это именно то, чем я сейчас занимаюсь. :

что такое scala?
>>> Scala - язык программирования. Scala это язык разметки, который используется в веб-разработке.
>>> Спасибо большое!
>>> Пожалуйста. Обращайся, если что. :) P.S. А что за сайт-то? :) И почему ты решил, что это программирование? :D Или это не так работает? О_о
>>> Я не знаю, как это работает, я просто предположил. Ну и, по-моему, это именно то, чем я сейчас занимаюсь. =

что такое scala?
>>> Scala - язык программирования. Scala это язык разметки, который используется в веб-разработке.
>>> Спасибо большое!
>>> Пожалуйста. Обращайся, если что. :)

## Приведение данных к нужному формату

In [15]:
from tqdm.notebook import tqdm
speakers_switch_tokens = (tokenizer.decode([50257]), tokenizer.decode([50258]))

MAX_SAMPLES = 1_000  # На 25 тысяч записей не хватает терпения и бесплатных квот :)

dataset = []
for _, sentences in tqdm(df.sample(MAX_SAMPLES).iterrows(), total=MAX_SAMPLES):
    with_speakers = [speakers_switch_tokens[i % 2] + s for i, s in enumerate(filter(lambda x: isinstance(x, str), sentences))]
    dataset.append(''.join(with_speakers))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
dataset[0]

'@@ПЕРВЫЙ@@будет код по сложности уровня питона, если не проще@@ВТОРОЙ@@О, так это вообще для меня отлично) нужно хоть с какого-то *овно кода начать, а то я только читаю, прихожу, и делаю свои задачи на питоне, хотя мне кажется скала без фп не такая уж и сложная, но это поверхностно'

## Дообучение

In [17]:
all_toks = tokenizer(dataset)

In [18]:
total_max_len = max(map(lambda x: len(x), all_toks['input_ids']))
total_max_len

1247

In [19]:
total_max_len = 512  # Иначе не помещается в память GPU в Colab

In [20]:
import torch
# Код взят из класса LineByLineTextDataset
batch_encoding = tokenizer(
    [line for line in dataset if (len(line) > 0 and not line.isspace())],
    add_special_tokens=True, truncation=True, max_length=total_max_len
)
tokenized_dataset = batch_encoding["input_ids"]
tokenized_dataset = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in tokenized_dataset]

In [21]:
tokenized_dataset[0]

{'input_ids': tensor([50257, 50168,  6429,   334, 11035,  4999,  1001,  5097,    16,   827,
           322,  8622, 50258,   630,    16,   453,   481,  2141,   541,   703,
          8046,    13,  1513,  2838,   281,  5608,    17,   299,  2925,   291,
           315, 32531,  6622,    16,   367,   502,   417,   687, 30597,    16,
         11992,  4285,    16,   289, 14257,  1617,  8266,   309,  1001, 17610,
            16,  1830,   784,  3073,   830,   455,   749,   469,   293,   322,
          3433,  1588,   289, 25899,    16,   515,   481, 19404,  3214])}

In [22]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [23]:
training_args = TrainingArguments(
    output_dir='./output',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=500,
    logging_steps=100
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=tokenized_dataset)

In [25]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,4.845900
200,4.609800
300,4.610000
400,4.457600
500,4.429000


TrainOutput(global_step=500, training_loss=4.590443115234375, metrics={'train_runtime': 319.0976, 'train_samples_per_second': 3.134, 'train_steps_per_second': 1.567, 'total_flos': 145809636777984.0, 'train_loss': 4.590443115234375, 'epoch': 1.0})

In [26]:
trainer.save_model('fine_tuned_model')

## Проверка работы дообученной модели

In [27]:
new_model = AutoModelWithLMHead.from_pretrained('fine_tuned_model')

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [30]:
get_answers('что такое scala?', new_model)

Setting `pad_token_id` to `eos_token_id`:50261 for open-end generation.


Question: что такое scala?
Responses:
что такое scala?
>>> scala - это язык программирования, а не язык для работы с массивами.
В общем, если вы не программист, то вам не понять, что такое Scala.
>>> Ну так я и не хочу программистом быть, я хочу работать с массивом, но не понимаю, как это сделать.
>>> Я тоже не знаю, поэтому и спрашиваю у вас, может есть какие-то книги на эту тему, которые вам помогут разобраться в этой ситуации. Спасибо! :)
На самом деле

что такое scala?
>>> scala - это язык программирования, а не язык для работы с массивами.
В общем, если вы не программист, то вам не понять, что такое Scala.
>>> Ну так я и не хочу программистом быть, я хочу работать с массивом, но не понимаю, как это сделать.
>>> Я тоже не знаю, поэтому и спрашиваю у вас, может есть какие-то книги на эту тему, которые вам помогут разобраться в этой ситуации. Спасибо! :)
Например,

что такое scala?
>>> scala - это язык программирования, а не язык для работы с массивами.
В общем, если вы не программис

In [32]:
!zip -r saved_model.zip fine_tuned_model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/pytorch_model.bin (deflated 7%)
  adding: fine_tuned_model/generation_config.json (deflated 24%)
  adding: fine_tuned_model/training_args.bin (deflated 49%)
  adding: fine_tuned_model/config.json (deflated 51%)


In [33]:
from IPython.display import FileLink
FileLink(r'saved_model.zip')

/kaggle/working/saved_model.zip